In [1]:
import streamlit as st
from pathlib import Path
from typing import Dict, Any, List, Tuple, Optional
from datetime import datetime
import tempfile
import asyncio
from functools import lru_cache
import time
from backend.kb_management.manager import KnowledgeBaseManager

import os

/home/asa/repo/git/A0580/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [18]:
# Configuration des chemins
storage_dir = "data/knowledge_bases/chromadb"


kb_manager = KnowledgeBaseManager(storage_directory=str(storage_dir))

In [2]:
import chromadb
client_dir = "data/knowledge_bases/chromadb/vector_storage"
client = chromadb.PersistentClient(path=client_dir)

In [8]:
collections = client.list_collections()
print(collections)

[Collection(id=493e41b9-23a5-4492-a3e1-a22e88cb9888, name=normes), Collection(id=a4d8d187-7854-4770-8850-1783be350aa0, name=CCTP_A0579), Collection(id=d766a265-2f58-4314-ab14-91529d9f45b7, name=A0275)]


In [21]:
collection = client.get_collection("normes")
result = collection.get(include=["embeddings"], limit=1)
dimension = len(result['embeddings'][0])
print(f"La dimension des embeddings de cette collection est : {dimension}")

IndexError: list index out of range

In [7]:
client.delete_collection("A0507")

In [4]:
kb_manager.list_knowledge_bases()

[{'id': 'normes',
  'title': 'normes ferroviaires',
  'description': 'règles de sécurité ferroviaires',
  'language': 'fr',
  'created_on': 1732717589},
 {'id': 'A0579',
  'title': 'Infrabel',
  'description': 'Affaire A0579 concernant le projet Infrabel, pour des locomotives pour la Belgique',
  'language': 'fr',
  'created_on': 1732717982}]

In [5]:
def list_chromadb_collections(self) -> List[dict]:
    """
    Liste toutes les collections ChromaDB avec leurs métadonnées.
    
    Returns:
        List[dict]: Liste des collections avec leurs informations
            - name: nom de la collection
            - dimension: dimension des vecteurs
            - metadata: métadonnées de la collection
            - count: nombre d'éléments
            - is_orphan: True si la collection n'a pas de KB correspondante
    """
    collections_info = []
    
    try:
        # Récupérer la liste des KBs valides
        valid_kb_ids = set(kb["id"] for kb in self.list_knowledge_bases())
        
        # Lister toutes les collections
        for collection_name in self.chroma_client.list_collections():
            try:
                collection = self.chroma_client.get_collection(collection_name)
                collection_info = collection.get()
                count = collection.count()
                
                # Vérifier si la collection est orpheline
                is_orphan = collection_name not in valid_kb_ids
                
                collections_info.append({
                    "name": collection_name,
                    "dimension": collection_info.get("dimension"),
                    "metadata": collection_info.get("metadata", {}),
                    "count": count,
                    "is_orphan": is_orphan,
                    "error": None
                })
            except Exception as e:
                # Gérer les collections corrompues
                collections_info.append({
                    "name": collection_name,
                    "dimension": None,
                    "metadata": {},
                    "count": 0,
                    "is_orphan": True,
                    "error": str(e)
                })
                
    except Exception as e:
        print(f"Erreur lors de la liste des collections: {str(e)}")
        
    return collections_info

In [6]:
print(list_chromadb_collections())

TypeError: list_chromadb_collections() missing 1 required positional argument: 'self'